In [1]:
## import (and install if necessary) pandas
%pip install pandas
import pandas as pd


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
df = pd.read_csv("full_data.csv"
                 ,parse_dates=['TripDate','StartDateTime','EndDateTime','CustomerBirthDate']
                 )
## process cost monetary data
df['Cost'] = df['Cost'].str.replace(',', '')
df['Cost'] = df['Cost'].str.replace('$', '')
df['Cost'] = df['Cost'].str.replace('(', '')
df['Cost'] = df['Cost'].str.replace(')', '')
df['Cost'] = df['Cost'].astype(float)
print(df.info())
#print(df.head(5))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1160755 entries, 0 to 1160754
Data columns (total 14 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   CustomerID         1160755 non-null  int64         
 1   CustomerSex        1160755 non-null  object        
 2   CustomerBirthDate  1160417 non-null  datetime64[ns]
 3   TripDate           1160755 non-null  datetime64[ns]
 4   VehicleID          1160755 non-null  int64         
 5   RunActualID        1160755 non-null  int64         
 6   StartDateTime      1160755 non-null  datetime64[ns]
 7   EndDateTime        1160755 non-null  datetime64[ns]
 8   PickupX            1110755 non-null  float64       
 9   PickupY            1110755 non-null  float64       
 10  DropoffX           1109028 non-null  float64       
 11  DropoffY           1109028 non-null  float64       
 12  Cost               1160538 non-null  float64       
 13  CollectedFareType  1147941 

In [3]:
## summary counts
runActualID_counts = df.groupby(['RunActualID'])['RunActualID'].count()
max_runActualIDs = runActualID_counts.max().item()
print(f"Maximum Number of rows per RunActualID: {max_runActualIDs}")
runActualID_vehicleID = df.groupby('RunActualID')['VehicleID'].nunique()
runActualIDs_more_than_one_vehicleID = runActualID_vehicleID[runActualID_vehicleID != 1]
print(f"Any RunActualIDs that have more than one VehicleID: {runActualIDs_more_than_one_vehicleID.nunique()}")
print(f"Unique number of RunActualIDs: {df['RunActualID'].nunique()}")
print(f"Unique number of CustomerIDs: {df['CustomerID'].nunique()}")
print(f"Unique number of VehicleIDs: {df['VehicleID'].nunique()}")

Maximum Number of rows per RunActualID: 18
Any RunActualIDs that have more than one VehicleID: 0
Unique number of RunActualIDs: 213272
Unique number of CustomerIDs: 10215
Unique number of VehicleIDs: 538


In [4]:
dates_per_runactualIDs = df.groupby(['RunActualID']).agg({'EndDateTime': 'max', 'StartDateTime':'min'})
dates_per_runactualIDs['EndDate'] = pd.to_datetime(dates_per_runactualIDs['EndDateTime'].dt.date)
dates_per_runactualIDs['StartDate'] = pd.to_datetime(dates_per_runactualIDs['StartDateTime'].dt.date)
dates_per_runactualIDs['diff'] = (dates_per_runactualIDs['EndDateTime'] - dates_per_runactualIDs['StartDateTime']).dt.total_seconds()
seconds_in_day = 86400
runactualIDs_over_day = dates_per_runactualIDs[dates_per_runactualIDs['diff'] > seconds_in_day]
#print(runactualIDs_over_day['RunActualId'].nunique())

print(f"""Number of RunActualIDs lasting over 24 hours: {runactualIDs_over_day.index.nunique()}
(Calculated by subtracting EndDateTime from StartDateTime)
Saving outlier RunActualIDs to `runactualIDs_over_day.csv""")
runactualIDs_over_day.to_csv("runactualIDs_over_day.csv", columns = ['StartDateTime','EndDateTime'])

Number of RunActualIDs lasting over 24 hours: 4
(Calculated by subtracting EndDateTime from StartDateTime)
Saving outlier RunActualIDs to `runactualIDs_over_day.csv


In [5]:
# adding in seconds per trip
df['TotalTripTimeSeconds'] = (df['EndDateTime'] - df['StartDateTime']).dt.total_seconds()


In [6]:
# sort by RunActualID and StartDateTime, add ride rank
df = df.sort_values(by=['RunActualID','StartDateTime'])
df = df.reset_index()
df['TripRank'] = df.sort_values(['RunActualID','StartDateTime'], ascending=True).groupby(['RunActualID']).cumcount() + 1


df_route_fields = df[['RunActualID','CustomerID','StartDateTime','PickupX','PickupY','EndDateTime','DropoffX','DropoffY','Cost','CollectedFareType','TotalTripTimeSeconds']]

In [ ]:
# generate field definitions
"""field_tuple = ()
for x in range(max_runActualIDs):
    y = x + 1
    field_tuple = field_tuple + (f"CustomerID_{y}=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"StartDateTime_{y}=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"PickupX_{y}=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"PickupY_{y}=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"EndDateTime_{y}=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"DropoffX_{y}=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"DropoffY_{y}=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"Cost_{y}=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"CollectedFareType_{y}=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"TotalTripTimeSeconds_{y}=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head({y}).tail(1))"
                                 )
print(field_tuple)
"""

In [8]:


# there is 100% a better way to do this, just not today:
df_routes = df_route_fields.groupby('RunActualID').agg(
    total_customers=pd.NamedAgg(column='CustomerID', aggfunc='count'),
    customerID_1=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(1).tail(1)),
    startdatetime_1=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(1).tail(1)),
    pickupx_1=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(1).tail(1)),
    pickupy_1=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(1).tail(1)),
    enddatetime_1=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(1).tail(1)),
    dropoffx_1=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(1).tail(1)),
    dropoffy_1=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(1).tail(1)),
    cost_1=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(1).tail(1)),
    collected_fare_type_1=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(1).tail(1)),
    total_trip_time_1=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(1).tail(1)),
    customerID_2=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(2).tail(1)),
    startdatetime_2=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(2).tail(1)),
    pickupx_2=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(2).tail(1)),
    pickupy_2=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(2).tail(1)),
    enddatetime_2=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(2).tail(1)),
    dropoffx_2=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(2).tail(1)),
    dropoffy_2=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(2).tail(1)),
    cost_2=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(2).tail(1)),
    collected_fare_type_2=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(2).tail(1)),
    total_trip_time_2=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(2).tail(1)),
    customerID_3=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(3).tail(1)),
    startdatetime_3=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(3).tail(1)),
    pickupx_3=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(3).tail(1)),
    pickupy_3=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(3).tail(1)),
    enddatetime_3=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(3).tail(1)),
    dropoffx_3=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(3).tail(1)),
    dropoffy_3=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(3).tail(1)),
    cost_3=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(3).tail(1)),
    collected_fare_type_3=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(3).tail(1)),
    total_trip_time_3=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(3).tail(1)),
    customerID_4=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(4).tail(1)),
    startdatetime_4=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(4).tail(1)),
    pickupx_4=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(4).tail(1)),
    pickupy_4=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(4).tail(1)),
    enddatetime_4=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(4).tail(1)),
    dropoffx_4=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(4).tail(1)),
    dropoffy_4=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(4).tail(1)),
    cost_4=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(4).tail(1)),
    collected_fare_type_4=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(4).tail(1)),
    total_trip_time_4=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(4).tail(1)),
    customerID_5=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(5).tail(1)),
    startdatetime_5=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(5).tail(1)),
    pickupx_5=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(5).tail(1)),
    pickupy_5=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(5).tail(1)),
    enddatetime_5=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(5).tail(1)),
    dropoffx_5=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(5).tail(1)),
    dropoffy_5=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(5).tail(1)),
    cost_5=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(5).tail(1)),
    collected_fare_type_5=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(5).tail(1)),
    total_trip_time_5=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(5).tail(1)),
    customerID_6=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(6).tail(1)),
    startdatetime_6=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(6).tail(1)),
    pickupx_6=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(6).tail(1)),
    pickupy_6=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(6).tail(1)),
    enddatetime_6=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(6).tail(1)),
    dropoffx_6=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(6).tail(1)),
    dropoffy_6=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(6).tail(1)),
    cost_6=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(6).tail(1)),
    collected_fare_type_6=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(6).tail(1)),
    total_trip_time_6=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(6).tail(1)),
    customerID_7=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(7).tail(1)),
    startdatetime_7=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(7).tail(1)),
    pickupx_7=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(7).tail(1)),
    pickupy_7=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(7).tail(1)),
    enddatetime_7=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(7).tail(1)),
    dropoffx_7=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(7).tail(1)),
    dropoffy_7=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(7).tail(1)),
    cost_7=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(7).tail(1)),
    collected_fare_type_7=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(7).tail(1)),
    total_trip_time_7=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(7).tail(1)),
    customerID_8=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(8).tail(1)),
    startdatetime_8=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(8).tail(1)),
    pickupx_8=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(8).tail(1)),
    pickupy_8=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(8).tail(1)),
    enddatetime_8=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(8).tail(1)),
    dropoffx_8=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(8).tail(1)),
    dropoffy_8=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(8).tail(1)),
    cost_8=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(8).tail(1)),
    collected_fare_type_8=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(8).tail(1)),
    total_trip_time_8=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(8).tail(1)),
    customerID_9=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(9).tail(1)),
    startdatetime_9=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(9).tail(1)),
    pickupx_9=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(9).tail(1)),
    pickupy_9=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(9).tail(1)),
    enddatetime_9=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(9).tail(1)),
    dropoffx_9=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(9).tail(1)),
    dropoffy_9=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(9).tail(1)),
    cost_9=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(9).tail(1)),
    collected_fare_type_9=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(9).tail(1)),
    total_trip_time_9=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(9).tail(1)),
    customerID_10=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(10).tail(1)),
    startdatetime_10=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(10).tail(1)),
    pickupx_10=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(10).tail(1)),
    pickupy_10=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(10).tail(1)),
    enddatetime_10=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(10).tail(1)),
    dropoffx_10=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(10).tail(1)),
    dropoffy_10=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(10).tail(1)),
    cost_10=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(10).tail(1)),
    collected_fare_type_10=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(10).tail(1)),
    total_trip_time_10=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(10).tail(1)),
    customerID_11=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(11).tail(1)),
    startdatetime_11=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(11).tail(1)),
    pickupx_11=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(11).tail(1)),
    pickupy_11=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(11).tail(1)),
    enddatetime_11=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(11).tail(1)),
    dropoffx_11=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(11).tail(1)),
    dropoffy_11=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(11).tail(1)),
    cost_11=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(11).tail(1)),
    collected_fare_type_11=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(11).tail(1)),
    total_trip_time_11=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(11).tail(1)),
    customerID_12=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(12).tail(1)),
    startdatetime_12=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(12).tail(1)),
    pickupx_12=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(12).tail(1)),
    pickupy_12=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(12).tail(1)),
    enddatetime_12=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(12).tail(1)),
    dropoffx_12=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(12).tail(1)),
    dropoffy_12=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(12).tail(1)),
    cost_12=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(12).tail(1)),
    collected_fare_type_12=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(12).tail(1)),
    total_trip_time_12=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(12).tail(1)),
    customerID_13=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(13).tail(1)),
    startdatetime_13=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(13).tail(1)),
    pickupx_13=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(13).tail(1)),
    pickupy_13=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(13).tail(1)),
    enddatetime_13=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(13).tail(1)),
    dropoffx_13=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(13).tail(1)),
    dropoffy_13=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(13).tail(1)),
    cost_13=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(13).tail(1)),
    collected_fare_type_13=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(13).tail(1)),
    total_trip_time_13=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(13).tail(1)),
    customerID_14=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(14).tail(1)),
    startdatetime_14=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(14).tail(1)),
    pickupx_14=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(14).tail(1)),
    pickupy_14=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(14).tail(1)),
    enddatetime_14=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(14).tail(1)),
    dropoffx_14=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(14).tail(1)),
    dropoffy_14=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(14).tail(1)),
    cost_14=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(14).tail(1)),
    collected_fare_type_14=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(14).tail(1)),
    total_trip_time_14=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(14).tail(1)),
    customerID_15=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(15).tail(1)),
    startdatetime_15=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(15).tail(1)),
    pickupx_15=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(15).tail(1)),
    pickupy_15=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(15).tail(1)),
    enddatetime_15=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(15).tail(1)),
    dropoffx_15=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(15).tail(1)),
    dropoffy_15=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(15).tail(1)),
    cost_15=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(15).tail(1)),
    collected_fare_type_15=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(15).tail(1)),
    total_trip_time_15=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(15).tail(1)),
    customerID_16=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(16).tail(1)),
    startdatetime_16=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(16).tail(1)),
    pickupx_16=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(16).tail(1)),
    pickupy_16=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(16).tail(1)),
    enddatetime_16=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(16).tail(1)),
    dropoffx_16=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(16).tail(1)),
    dropoffy_16=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(16).tail(1)),
    cost_16=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(16).tail(1)),
    collected_fare_type_16=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(16).tail(1)),
    total_trip_time_16=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(16).tail(1)),
    customerID_17=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(17).tail(1)),
    startdatetime_17=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(17).tail(1)),
    pickupx_17=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(17).tail(1)),
    pickupy_17=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(17).tail(1)),
    enddatetime_17=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(17).tail(1)),
    dropoffx_17=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(17).tail(1)),
    dropoffy_17=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(17).tail(1)),
    cost_17=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(17).tail(1)),
    collected_fare_type_17=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(17).tail(1)),
    total_trip_time_17=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(17).tail(1)),
    customerID_18=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(18).tail(1)),
    startdatetime_18=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(18).tail(1)),
    pickupx_18=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(18).tail(1)),
    pickupy_18=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(18).tail(1)),
    enddatetime_18=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(18).tail(1)),
    dropoffx_18=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(18).tail(1)),
    dropoffy_18=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(18).tail(1)),
    cost_18=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(18).tail(1)),
    collected_fare_type_18=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(18).tail(1)),
    total_trip_time_18=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(18).tail(1))
)
    
    
df_routes.to_csv("wide_raw_backup.csv")

In [10]:
# clearing out duplicate values

df_routes.loc[df_routes['total_customers'] < 18 , ['customerID_18','startdatetime_18','pickupx_18','pickupy_18','enddatetime_18','dropoffx_18','dropoffy_18','cost_18','collected_fare_type_18','total_trip_time_18']] = pd.NA
df_routes.loc[df_routes['total_customers'] < 17 , ['customerID_17','startdatetime_17','pickupx_17','pickupy_17','enddatetime_17','dropoffx_17','dropoffy_17','cost_17','collected_fare_type_17','total_trip_time_17']] = pd.NA
df_routes.loc[df_routes['total_customers'] < 16 , ['customerID_16','startdatetime_16','pickupx_16','pickupy_16','enddatetime_16','dropoffx_16','dropoffy_16','cost_16','collected_fare_type_16','total_trip_time_16']] = pd.NA
df_routes.loc[df_routes['total_customers'] < 15 , ['customerID_15','startdatetime_15','pickupx_15','pickupy_15','enddatetime_15','dropoffx_15','dropoffy_15','cost_15','collected_fare_type_15','total_trip_time_15']] = pd.NA
df_routes.loc[df_routes['total_customers'] < 14 , ['customerID_14','startdatetime_14','pickupx_14','pickupy_14','enddatetime_14','dropoffx_14','dropoffy_14','cost_14','collected_fare_type_14','total_trip_time_14']] = pd.NA
df_routes.loc[df_routes['total_customers'] < 13 , ['customerID_13','startdatetime_13','pickupx_13','pickupy_13','enddatetime_13','dropoffx_13','dropoffy_13','cost_13','collected_fare_type_13','total_trip_time_13']] = pd.NA
df_routes.loc[df_routes['total_customers'] < 12 , ['customerID_12','startdatetime_12','pickupx_12','pickupy_12','enddatetime_12','dropoffx_12','dropoffy_12','cost_12','collected_fare_type_12','total_trip_time_12']] = pd.NA
df_routes.loc[df_routes['total_customers'] < 11 , ['customerID_11','startdatetime_11','pickupx_11','pickupy_11','enddatetime_11','dropoffx_11','dropoffy_11','cost_11','collected_fare_type_11','total_trip_time_11']] = pd.NA
df_routes.loc[df_routes['total_customers'] < 10 , ['customerID_10','startdatetime_10','pickupx_10','pickupy_10','enddatetime_10','dropoffx_10','dropoffy_10','cost_10','collected_fare_type_10','total_trip_time_10']] = pd.NA
df_routes.loc[df_routes['total_customers'] < 9 , ['customerID_9','startdatetime_9','pickupx_9','pickupy_9','enddatetime_9','dropoffx_9','dropoffy_9','cost_9','collected_fare_type_9','total_trip_time_9']] = pd.NA
df_routes.loc[df_routes['total_customers'] < 8 , ['customerID_8','startdatetime_8','pickupx_8','pickupy_8','enddatetime_8','dropoffx_8','dropoffy_8','cost_8','collected_fare_type_8','total_trip_time_8']] = pd.NA
df_routes.loc[df_routes['total_customers'] < 7 , ['customerID_7','startdatetime_7','pickupx_7','pickupy_7','enddatetime_7','dropoffx_7','dropoffy_7','cost_7','collected_fare_type_7','total_trip_time_7']] = pd.NA
df_routes.loc[df_routes['total_customers'] < 6 , ['customerID_6','startdatetime_6','pickupx_6','pickupy_6','enddatetime_6','dropoffx_6','dropoffy_6','cost_6','collected_fare_type_6','total_trip_time_6']] = pd.NA
df_routes.loc[df_routes['total_customers'] < 5 , ['customerID_5','startdatetime_5','pickupx_5','pickupy_5','enddatetime_5','dropoffx_5','dropoffy_5','cost_5','collected_fare_type_5','total_trip_time_5']] = pd.NA
df_routes.loc[df_routes['total_customers'] < 4 , ['customerID_4','startdatetime_4','pickupx_4','pickupy_4','enddatetime_4','dropoffx_4','dropoffy_4','cost_4','collected_fare_type_4','total_trip_time_4']] = pd.NA
df_routes.loc[df_routes['total_customers'] < 3 , ['customerID_3','startdatetime_3','pickupx_3','pickupy_3','enddatetime_3','dropoffx_3','dropoffy_3','cost_3','collected_fare_type_3','total_trip_time_3']] = pd.NA
df_routes.loc[df_routes['total_customers'] < 2 , ['customerID_2','startdatetime_2','pickupx_2','pickupy_2','enddatetime_2','dropoffx_2','dropoffy_2','cost_2','collected_fare_type_2','total_trip_time_2']] = pd.NA

In [11]:
# output wide results to csv
df_routes.head(5).to_csv("processed_aar_trips_wide.csv", index=False)

In [13]:
# output long results to csv
header = ['RunActualID','VehicleID','CustomerID','CustomerSex','CustomerBirthDate','TripRank','TripDate','StartDateTime','EndDateTime','TotalTripTimeSeconds','Cost','CollectedFareType','PickupX','PickupY','DropoffX','DropoffY']
df.head(5).to_csv("processed_aar_trips_long.csv", index=False, columns=header)

In [18]:
# output customer key to csv
print(df['CustomerID'].nunique())
customer_key = df.groupby(['CustomerID']).agg(
    CustomerSex=pd.NamedAgg(column='CustomerSex', aggfunc='max'),
    CustomerBirthDate=pd.NamedAgg(column='CustomerBirthDate', aggfunc='max'),
    TotalTrips=pd.NamedAgg(column='CustomerID', aggfunc='count')
    )
customer_key.to_csv("customer_key.csv")

10215
